<a href="https://colab.research.google.com/github/MathiLogha/GoogleNet/blob/main/GoogleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision
import os
import pandas as pd
from tqdm import tqdm
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean
from torch.utils.data import (
    Dataset,
    DataLoader,
)

In [ ]:
!pip install torch-summary
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class ImagesDataProcess(Dataset):
    def __init__(self,csv_file,root_dir,tranform=None):
        self.annotation=pd.read_csv(csv_file)
        self.root_dir=root_dir
        self.tranform=tranform

    def __len__(self):
        return len(self.annotation)

    def __getitem__(self,index):
        img_path=os.path.join(self.root_dir,self.annotation.iloc[index,0])
        image=io.imread(img_path)
        image=resize(image,(250,250),anti_aliasing=True)
        y_label=torch.tensor(int(self.annotation.iloc[index,1]))

        if self.tranform:
            image=self.tranform(image)
        return (image,y_label)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving riceClassification.csv to riceClassification.csv


In [ ]:
in_channel = 3
num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10
root_dir_path = os.path.join(r"E:\anconda\envs\new_env\Rice_Image_Dataset")
csv_file_path = "riceClassification.csv"

In [ ]:
dataset=ImagesDataProcess(
    csv_file=csv_file_path,
    root_dir=root_dir_path,
    tranform=transforms.ToTensor()
)

In [ ]:
len(dataset)

18185

In [ ]:
train_set, test_set = torch.utils.data.random_split(dataset, [17000, 1185])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

In [ ]:
print(enumerate(tqdm(train_loader)))

  0%|          | 0/532 [00:00<?, ?it/s]

In [ ]:
model = torchvision.models.googlenet(pretrained=True)
model.fc=nn.Sequential(
    nn.Linear(in_features=1024,out_features=512),
    nn.ReLU(),
    nn.Linear(in_features=512,out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128,out_features=32),
    nn.ReLU(),
    nn.Linear(in_features=32,out_features=2,bias=True)
)
model.to(device);
model

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 81.1MB/s]


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [ ]:
from torchsummary import summary
print(summary(model, (3, 250, 250),verbose=0))

Layer (type:depth-idx)                   Output Shape              Param #
├─BasicConv2d: 1-1                       [-1, 64, 125, 125]        --
|    └─Conv2d: 2-1                       [-1, 64, 125, 125]        9,408
|    └─BatchNorm2d: 2-2                  [-1, 64, 125, 125]        128
├─MaxPool2d: 1-2                         [-1, 64, 62, 62]          --
├─BasicConv2d: 1-3                       [-1, 64, 62, 62]          --
|    └─Conv2d: 2-3                       [-1, 64, 62, 62]          4,096
|    └─BatchNorm2d: 2-4                  [-1, 64, 62, 62]          128
├─BasicConv2d: 1-4                       [-1, 192, 62, 62]         --
|    └─Conv2d: 2-5                       [-1, 192, 62, 62]         110,592
|    └─BatchNorm2d: 2-6                  [-1, 192, 62, 62]         384
├─MaxPool2d: 1-5                         [-1, 192, 31, 31]         --
├─Inception: 1-6                         [-1, 256, 31, 31]         --
|    └─BasicConv2d: 2-7                  [-1, 64, 31, 31]          --
|

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

**GoogleNet:**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

In [ ]:
model = models.googlenet(pretrained=True)
num_classes = 3
in_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(in_features=1024,out_features=512),
    nn.ReLU(),
    nn.Linear(in_features=512,out_features=128),
    nn.ReLU(),
    nn.Linear(in_features=128,out_features=32),
    nn.ReLU(),
    nn.Linear(in_features=32,out_features=3,bias=True)
)
#model.to(device);
model
#model.fc = nn.Linear(in_features, num_classes)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

In [ ]:
test_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/Rice', transform=train_transforms)
test_dataset = datasets.ImageFolder('/content/drive/MyDrive/Rice', transform=test_transforms)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=5, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=5, shuffle=False)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, epoch_loss))

Epoch [1/10], Loss: 1.1010
Epoch [2/10], Loss: 1.0978
Epoch [3/10], Loss: 1.0969
Epoch [4/10], Loss: 1.0967
Epoch [5/10], Loss: 1.0909
Epoch [6/10], Loss: 1.1012
Epoch [7/10], Loss: 1.1046
Epoch [8/10], Loss: 1.0997
Epoch [9/10], Loss: 1.0966
Epoch [10/10], Loss: 1.0928


**GoogleNet with one layer:**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

In [ ]:
model = models.googlenet(pretrained=True)
num_classes = 3
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

In [ ]:
test_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/Rice', transform=train_transforms)
test_dataset = datasets.ImageFolder('/content/drive/MyDrive/Rice', transform=test_transforms)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=5, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=5, shuffle=False)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, epoch_loss))

Epoch [1/10], Loss: 1.1956
Epoch [2/10], Loss: 1.1800
Epoch [3/10], Loss: 1.1118
Epoch [4/10], Loss: 0.9812
Epoch [5/10], Loss: 1.0430
Epoch [6/10], Loss: 1.0390
Epoch [7/10], Loss: 0.9705
Epoch [8/10], Loss: 1.0801
Epoch [9/10], Loss: 0.9824
Epoch [10/10], Loss: 0.9003
